In [ ]:
import pandas as pd
import psycopg2
import json
from tqdm import tqdm

In [ ]:
food = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/09.15_restaurant.csv",index_col=0)
food.head(3)

In [ ]:
food.columns

Index(['contentsid', 'title', 'roadaddress', 'tag', 'introduction', 'latitude',
       'longitude', 'contentscd.label', 'phoneno', 'repPhoto.photoid.imgpath',
       'repPhoto.photoid.thumbnailpath', '인기점수', 'tag_len', 'popular_tag',
       'len'],
      dtype='object')

In [ ]:
food.shape

(1623, 15)

In [ ]:
id_list = [i for i in range(1,1624)]
food['id'] = id_list

In [ ]:
food = food[['id','title', 'roadaddress', 'tag', 'introduction', 'latitude',
       'longitude','phoneno', 'repPhoto.photoid.imgpath','인기점수','popular_tag']]

In [ ]:
food = food.rename(columns = {'repPhoto.photoid.imgpath':"image_url"})

In [ ]:
food.head(2)

,id,title,roadaddress,tag,introduction,latitude,longitude,phoneno,image_url,인기점수,popular_tag
4051,1,제주김만복(만복이네김밥집),제주특별자치도 제주시 오라로 41,"['전복김밥', '통전복주먹밥', '미역국', '음식', '해물라면', '라면']","전복내장으로 만든 김밥, 심플한 메뉴로 많은 여행객들에게 사랑받는 곳",33.497049,126.508731,064-759-8582,https://api.cdn.visitjeju.net/photomng/imgpath...,144324.0,"음식,라면,해물라면"
3548,2,봄날카페,제주특별자치도 제주시 애월읍 애월로1길 25,"['카페', '커피', '음료', '음식', '콜드브루', '아메리카노', '바닐라...",드라마 '맨도롱 또똣'의 주인공이 일하던 카페,33.462460,126.309590,064-799-4999,https://api.cdn.visitjeju.net/photomng/imgpath...,121065.0,"아메리카노,카페모카,바닐라라떼,카푸치노,음료,카페,커피,음식,카페라떼"


In [ ]:
food['tag'] = food['tag'].apply(lambda x: x[1:-1])

In [ ]:
food['tag'] = food['tag'].apply(lambda x: x.split(','))

In [ ]:
food['tag']

4051    ['전복김밥',  '통전복주먹밥',  '미역국',  '음식',  '해물라면',  '...
3548    ['카페',  '커피',  '음료',  '음식',  '콜드브루',  '아메리카노',...
3990                      ['고기국수',  '한식',  '향토음식',  '음식']
3505    ['전복',  '돌솥밥',  '회',  '음식',  '전복해물탕',  '전복죽', ...
4170    ['카페',  '커피',  '음료',  '디저트',  '음식',  '아메리카노', ...
                              ...                        
2172             ['정육식당',  '노형동',  '함흥식냉면',  '음식',  '식당']
2439         ['베라체',  '파스타',  '리조또',  '피자',  '음식',  '식당']
3172                 ['삼화',  '곰탕',  '나주곰탕',  '음식',  '식당']
3486        ['즉석떡볶이',  '차돌떡볶이',  '통큰오짱떡볶이',  '음식',  '카페']
4055                   ['협재',  '보말',  '전복',  '음식',  '식당']
Name: tag, Length: 1623, dtype: object

In [ ]:
food.sample()['tag'].values[0]

["'차'", " '밀크티'", " '카카오티'", " '음식'"]

## ' 제거하기 위해서 이전 파일 가져와서 Tag 합치기

In [ ]:
tour = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/tourist_attraction_with_rating_sort_by_type.csv",index_col = 0)
restaurant = tour.loc[tour['contentscd.label']=='음식점']

In [ ]:
food['tag'] = restaurant['tag']

In [ ]:
food.head(4)

,id,title,roadaddress,tag,introduction,latitude,longitude,phoneno,image_url,인기점수,popular_tag
4051,1,제주김만복(만복이네김밥집),제주특별자치도 제주시 오라로 41,"전복김밥,통전복주먹밥,미역국,음식,해물라면,라면","전복내장으로 만든 김밥, 심플한 메뉴로 많은 여행객들에게 사랑받는 곳",33.497049,126.508731,064-759-8582,https://api.cdn.visitjeju.net/photomng/imgpath...,144324.0,"음식,라면,해물라면"
3548,2,봄날카페,제주특별자치도 제주시 애월읍 애월로1길 25,"카페,커피,음료,음식,콜드브루,아메리카노,바닐라라떼,카페라떼,아인슈페너,카푸치노,카...",드라마 '맨도롱 또똣'의 주인공이 일하던 카페,33.462460,126.309590,064-799-4999,https://api.cdn.visitjeju.net/photomng/imgpath...,121065.0,"아메리카노,카페모카,바닐라라떼,카푸치노,음료,카페,커피,음식,카페라떼"
3990,3,올래국수,제주특별자치도 제주시 귀아랑길 24 (연동),"고기국수,한식,향토음식,음식",제주 향토 음식인 고기국수 전문점이다. 2016 KOREAT JEJU TOP 10 선정,33.491500,126.497300,064-742-7355,https://api.cdn.visitjeju.net/photomng/imgpath...,119913.0,"한식,음식,향토음식,고기국수"
3505,4,명진전복,제주특별자치도 제주시 구좌읍 해맞이해안로 1282,"전복,돌솥밥,회,음식,전복해물탕,전복죽,전복뚝배기,전복버터구이,전복물회",제주도 대표 전복요리 전문점 2016 KOREAT JEJU TOP10 선정,33.532420,126.849850,064-782-9944,https://api.cdn.visitjeju.net/photomng/imgpath...,92336.0,"전복죽,전복물회,회,음식,전복뚝배기,전복"


###iloc 활용을 위해서 index 초기화 해주기

In [ ]:
food = food.reset_index(drop=True)

In [ ]:
food.to_csv("/content/drive/MyDrive/yeoreodigm/data_files/09.28_restaurant.csv")

In [ ]:
food['tag_list'] = food['tag'].apply(lambda x: x.split(','))

In [ ]:
food['num_of_tag'] = food['tag_list'].apply(lambda x: len(x))

In [ ]:
food.head(3)

,id,title,roadaddress,tag,introduction,latitude,longitude,phoneno,image_url,인기점수,popular_tag,tag_list,num_of_tag
4051,1,제주김만복(만복이네김밥집),제주특별자치도 제주시 오라로 41,"전복김밥,통전복주먹밥,미역국,음식,해물라면,라면","전복내장으로 만든 김밥, 심플한 메뉴로 많은 여행객들에게 사랑받는 곳",33.497049,126.508731,064-759-8582,https://api.cdn.visitjeju.net/photomng/imgpath...,144324.0,"음식,라면,해물라면","[전복김밥, 통전복주먹밥, 미역국, 음식, 해물라면, 라면]",6
3548,2,봄날카페,제주특별자치도 제주시 애월읍 애월로1길 25,"카페,커피,음료,음식,콜드브루,아메리카노,바닐라라떼,카페라떼,아인슈페너,카푸치노,카...",드라마 '맨도롱 또똣'의 주인공이 일하던 카페,33.462460,126.309590,064-799-4999,https://api.cdn.visitjeju.net/photomng/imgpath...,121065.0,"아메리카노,카페모카,바닐라라떼,카푸치노,음료,카페,커피,음식,카페라떼","[카페, 커피, 음료, 음식, 콜드브루, 아메리카노, 바닐라라떼, 카페라떼, 아인슈...",13
3990,3,올래국수,제주특별자치도 제주시 귀아랑길 24 (연동),"고기국수,한식,향토음식,음식",제주 향토 음식인 고기국수 전문점이다. 2016 KOREAT JEJU TOP 10 선정,33.491500,126.497300,064-742-7355,https://api.cdn.visitjeju.net/photomng/imgpath...,119913.0,"한식,음식,향토음식,고기국수","[고기국수, 한식, 향토음식, 음식]",4


##상위 300개 태그를 분류하자....

In [ ]:
top500_tag_list = restaurant['tag'].str.get_dummies(sep=',').mean().sort_values(ascending=False)[:500].index

In [ ]:
top100 = top500_tag_list[:100]
top200 = top500_tag_list[100:200]
top300 = top500_tag_list[200:300]
top400 = top500_tag_list[300:400]
top500 = top500_tag_list[400:]

In [ ]:
top100

Index(['음식', '카페', '식당', '아메리카노', '커피', '한식', '흑돼지', '카페라떼', '디저트', '갈치조림',
       '빵', '회', '음료', '에이드', '오겹살', '김치찌개', '파스타', '제주갈치조림', '흑돼지구이', '해산물',
       '바닐라라떼', '갈치구이', '고등어구이', '전복죽', '횟집', '향토음식', '옥돔구이', '베이커리', '고등어조림',
       '샌드위치', '전복뚝배기', '브런치', '물회', '전복', '에스프레소', '피자', '한치', '성게미역국', '케이크',
       '정식', '제주갈치구이', '고기국수', '돼지구이', '고등어회', '라면', '칼국수', '삼겹살', '카페모카',
       '돈까스', '볶음밥', '목살', '한치물회', '샐러드', '녹차라떼', '된장찌개', '아이스크림', '크림파스타',
       '갈치', '해물라면', '돔베고기', '카푸치노', '해물뚝배기', '중식', '짬뽕', '두루치기', '전복물회', '커플',
       '해장국', '주스', '양식', '일식', '과일주스', '차', '스무디', '갈비', '매운탕', '친구', '휴식/힐링',
       '국밥', '짜장면', '치킨', '스테이크', '생선구이정식', '몸국', '고등어', '탕수육', '회덮밥', '빙수',
       '떡볶이', '핸드드립커피', '김밥', '초밥', '갈치회', '해물탕', '양념갈비', '빵지순례', '냉면', '생선조림',
       '화덕피자', '보말칼국수'],
      dtype='object')

In [ ]:
top200

Index(['맥주', '광어회', '고르곤졸라화덕피자', '우동', '부모', '문어숙회', '햄버거', '갈비탕', '비빔밥',
       '밀크티', '추자도', '쿠키', '방어', '딸기라떼', '레몬에이드', '수제맥주', '순두부찌개', '치킨샐러드',
       '성게국', '분식', '항정살', '새우볶음밥', '팥빙수', '가브리살', '해물파스타', '방어회', '비빔국수',
       '감자튀김', '전복버터구이', '미트볼파스타', '옥돔', '멸치국수', '우도', '해물짬뽕', '갈치국', '소라',
       '와인', '전복해물탕', '아이', '서귀포', '회국수', '빵집', '오일파스타', '청귤차', '돼지고기',
       '추자음식점', '리조또', '한치회', '활어회', '돈가스', '육회', '뷔페', '리코타치즈샐러드', '추자식당',
       '근고기', '수육', '새우튀김', '수제버거', '문어', '샤브샤브', '덮밥', '성게비빔밥', '제육볶음', '만두',
       '치즈케이크', '새우로제파스타', '우럭매운탕', '내장탕', '에그타르트', '삼계탕', '봉골레파스타', '보말죽',
       '구제주', '스시', '아침식사', '치즈돈까스', '카레', '수제돈까스', '자몽에이드', '백숙', '갈비찜',
       '콩국수', '망고빙수', '청국장', '포토스팟', '펍', '전복구이', '대패삼겹살', '신제주', '순대', '티라미수',
       '사계절', '해물파전', '국수', '드립커피', '와플', '아이스티', '아인슈페너', '순대국밥', '레스토랑'],
      dtype='object')

In [ ]:
top300

Index(['오분작뚝배기', '딸기스무디', '감바스', '망고주스', '버거', '혼자', '망고스무디', '스파게티', '소라물회',
       '말고기', '쌀국수', '전복돌솥밥', '백반', '토스트', '우럭조림', '성게국수', '냉소바', '다금바리',
       '햄버거스테이크', '애월', '생크림케익', '콜드브루', '간장게장', '눈꽃빙수', '돌체라떼', '조림',
       '돼지김치찌개', '올레시장', '딱새우', '불고기', '망고', '망고에이드', '보말', '바게트', '칵테일',
       '해물찜', '당근케이크', '홍차', '육개장', '가정식', '제주카페', '분짜', '한정식', '디저트카페',
       '생선구이', '페페로니햄피자', '눈꽃우유빙수', '돌솥밥', '함덕해수욕장', '자리물회', '함박스테이크', '즉석떡볶이',
       '수제청', '모듬회', '베이글', '튀김', '카라멜마끼아또', '굴비정식', '소고기쌀국수', '뚝배기', '떡갈비',
       '떡', '횔어우럭매운탕', '연탄구이', '닭볶음탕', '전골', '오메기떡', '경관/포토', '휴식/치유', '녹차',
       '치즈버거', '한림', '닭칼국수', '모닥치기', '제주시청', '보말국', '잡채밥', '성게', '마르게리타화덕피자',
       '핸드드립', '헤이즐넛라떼', '양념치킨', '고기짬뽕', '문어라면', '요거트', '꽃차', '해물전골', '숲', '죽',
       '허니브레드', '우수관광사업체', '막걸리', '동문시장', '치아바타', '애플망고스무디', '오션뷰', '안전여행스탬프',
       '문어물회', '술', '수제마카롱'],
      dtype='object')

In [ ]:
top400

Index(['파전', '땅콩아이스크림', '소머리국밥', '한우구이', '전복회', '한우', '협재', '오리고기', '요거트스무디',
       '해물', '해물짜장', '퓨전', '무인카페', '통닭', '우도땅콩아이스크림', '수제비', '핫도그', '수제청음료',
       '순대국', '왕만두', '중문', '성산', '타르트', '라떼', '보쌈', '도시락', '성산일출봉', '쌈밥',
       '차돌박이', '망고요거트', '젤라또', '땡초김밥', '마늘바게트', '애월해안도로', '북카페', '라멘',
       '블루베리스무디', '생과일주스', '표선', '통갈치', '지리', '아보카도샐러드', '소고기', '마라탕', '들깨수제비',
       '청년', '마카롱', '프레즐', '찻집', '설렁탕', '참치김밥', '해물칼국수', '얼그레이', '숯불구이',
       '전복갈비탕', '가정식카레', '순두부', '노형동', '메밀', '닭도리탕', '식빵', '전통주', '나베',
       '전복해물뚝배기', '멸치김밥', '꽃등심스테이크', '구이', '치킨가라아게', '노형', '자연산', '무슬림',
       '통갈치구이', '한라산볶음밥', '반미', '제과', '생맥주', '장어덮밥', '연어덮밥', '아라동', '김치김밥',
       '호프', '동태찌개', '흑돼지돈까스', '우유식빵', '광어', '딸기아이스크림빙수', '애월맛집', '꼼장어', '간짜장',
       '고사리육개장', '라자냐', '팟타이', '고사리', '낙지볶음', '전복어', '오미자주스', '곡물라떼', '삼치회',
       '곰탕', '빙떡'],
      dtype='object')

In [ ]:
top500

Index(['로컬식당', '아포가토', '로컬푸드', '바나나팬케이크', '연동', '함덕', '토종닭', '타코', '피쉬앤칩스',
       '제주맛집', '보리빵', '흑돼지오겹살', '맑음', '후라이드치킨', '감자탕', '망고음료', '신화월드',
       '토마호크스테이크', '한라봉', '대정', '제주신화월드맛집', '도미지리', '보말미역국', '한라수목원', '한라봉에이드',
       '더블치즈김밥', '말고기사시미', '제주빵집', '초벌구이', '초코파운드', '경양식돈가스', '원두', '유기농',
       '초콜릿', '밤식빵', '외도', '양갱', '추자도음식점', '참돔회', '티', '치즈떡볶이', '유기농빵', '뷰맛집',
       '연북로', '일몰', '캐모마일', '우럭', '흑돼지버거', '어묵탕', '크림치즈', '막국수', '유자에이드',
       '사시미', '뼈다귀탕', '머리고기', '열무국수', '비자림', '츄러스', '크로와상', '테마공원', '치킨리조또',
       '베이커리카페', '마늘치킨', '크로플', '바지락칼국수', '유부초밥', '말육회', '추자굴비정식', '감자고로케',
       '콩나물국밥', '오리백숙', '추어탕', '고기만두', '흑돼지맛집', '망고쉐이크', '간식', '밀면', '옻닭',
       '브런치카페', '파니니', '소고기해장국', '제주시', '퓨전음식', '돌돔', '정육식당', '굴전', '구좌',
       '표선해수욕장', '자연산회', '혼밥', '조기구이', '닭개장칼국수', '국물닭발', '수제초콜릿', '닭강정', '그라탕',
       '아구찜', '김치찜', '굴', '핑크뮬리'],
      dtype='object')

In [ ]:
korean = "음식 식당 한식 흑돼지 갈치조림 오겹살 김치찌개 제주갈치조림 흑돼지구이 해산물 갈치구이 고등어구이 전복죽 향토음식 옥돔구이 고등어조림 전복뚝배기 물회 전복 한치 성게미역국 정식 제주갈치구이 고기국수 돼지구이 라면 칼국수 삼겹살 볶음밥 목살 한치물회 된장찌개 갈치 해물라면 돔베고기 해물뚝배기 두루치기 전복물회 해장국 갈비 매운탕 국밥 생선구이정식 몸국 고등어 갈치회 해물탕 양념갈비 냉면 생선조림 보말칼국수 광어회 문어숙회 갈비탕 비빔밥 방어 순두부찌개 성게국 항정살 가브리살 비빔국수 옥돔 멸치국수 갈치국 소라 전복해물탕 회국수 돼지고기 한치회 활어회 육회 근고기 문어 성게비빔밥 제육볶음 우럭매운탕 내장탕 삼계탕 보말죽 수제돈까스 백숙 갈비찜 콩국수 청국장 대패삼겹살 해물파전 국수 순대국밥 오분작뚝배기 소라물회 말고기 쌀국수 전복돌솥밥 백반 우럭조림 성게국수 다금바리 간장게장 조림 돼지김치찌개 불고기 해물찜 육개장 한정식 생선구이 돌솥밥 자리물회 굴비정식 소고기쌀국수 뚝배기 떡갈비 떡 활어우럭매운탕 연탄구이 닭볶음탕 문어라면 파전 소머리국밥 한우구이 전복회 한우 오리고기 통닭 수제비 순대국 보쌈 쌈밥 차돌박이 ㄷ통갈치 지리 소고기 들깨수제비 설렁탕 해물칼국수 숯불구이 전복갈비탕 순두부 메밀 닭도리탕 전복해물뚝배기 통갈치구이 한라산볶음밥 꼼장어 동태찌개 고사리육개장 고사리 낙지볶음 전복어 곰탕 토종닭 흑돼지오겹살 도미지리 보말미역국 초벌구이 우럭 뼈다귀탕 머리고기 열무국수 바지락칼국수 말육회 콩나물국밥 오리백숙 추어탕 흑돼지맛집 밀면 소고기해장국 굴전 조기구이 닭개장칼국수 국물닭발 아구찜 김치찜 김치찌개 찌개 찌개전문점 굴 막국수"

In [ ]:
japanese = "음식 식당 회 횟집 일식 초밥 스시 광어회 문어숙회 방어 방어회 한치회 활어회 돈가스 덮밥 새우튀김 딱새우 우동 돈까스 치즈돈까스 수제돈까스 스시전문점 일식집 치킨가라아게 장어덮밥 연어덮밥 아라동 사케 삼치회 가라아게 경양식돈가스 참돔회 사시미 유부초밥 자연산회 라멘 돈코츠라멘 돈코츠 소유라멘 소유" 

In [ ]:
chinese =  "음식 식당 짬뽕 중식 짜장면 볶음밥 탕수육 마라탕 마라 새우볶음밥 해물짬뽕 고기짬뽕 해물짜장 삼선짜장 삼선짬뽕 왕만두 중국 간짜장 깐풍기 고기만두"

In [ ]:
western = "음식 식당 파스타 피자 햄버거 브런치 크림파스타 샐러드 양식 스테이크 화덕피자 고르곤졸라화덕피자 해물파스타 감자튀김 와인 미트볼파스타 오일파스타 리코타치즈샐러드 리조또 수제버거 새우로제파스타 봉골레파스타 펍 레스토랑 감바스 버거 스파게티 햄버거스테이크 페페로니햄피자 함박스테이크 치즈버거 마르게리타화덕피자 토스트 아보카도샐러드 꽃등심스테이크 무슬림 라자냐 팟타이 타코 피쉬앤칩스 파니니 흑돼지버거 치킨리조또 그라탕"

In [ ]:
cafe = "음식 카페 아메리카노 커피 카페라떼 디저트 빵 음료 에이드 바닐라라떼 베이커리 샌드위치 에스프레소 케이크 카페모카 샐러드 녹차라떼 아이스크림 카푸치노 주스 과일주스 차 스무디 빙수 핸드드립커피 밀크티 쿠키 딸기라떼 레몬에이드 팥빙수 빵집 리코타치즈샐러드 치즈케이크 자몽에이드 망고빙수 티라미수 와플 아이스티 딸기스무디 망고주스 망고스무디 생크림케익 콜드브루 눈꽃빙수 돌체라떼 망고 망고에이드 바게트 홍차 당근케이크 눈꽃우유빙수 수제청 베이글 카라멜마끼아또 떡 오메기떡 핸드드립 허니브레드 헤이즐넛라떼 요거트 꽃차 땅콩아이스크림 요거트스무디 우도땅콩아이스크림 핫도그 수제청음료 타르트 라떼 망고요거트 젤라또 마늘바게트 블루베리스무디 생과일주스 아보카도샐러드 얼그레이 찻집 프레즐 마카롱 반미 제과 우유식빵 딸기아이스크림빙수 빙수 과일빙수 오미자주스 곡물라떼 빙떡 아포가토 망고음료 초코파운드 제주빵집 원두 초콜릿 밤식빵 양갱 티 유기농빵 캐모마일 크림치즈 유자에이드 감자고로케 크로플 베이커리카페 망고쉐이크 간식 브런치카페 수제초콜릿"

In [ ]:
bunsik = "음식 떡볶이 김밥 순대 분식 치즈 떡볶이 닭강정"

###태그갯수가 부족한 애들 확인해보기

In [ ]:
food[food['num_of_tag']<5].shape

(87, 14)

##String -> set 전환 후 교집합 비율로 식당 구분하기

In [ ]:
korean = korean.split(sep=' ')
japanese = japanese.split(sep=' ')
chinese = chinese.split(sep=' ')
western = western.split(sep= ' ')
bunsik = bunsik.split(sep=' ')
cafe = cafe.split(sep=' ')

In [ ]:
korean[:10]

['음식', '식당', '한식', '흑돼지', '갈치조림', '오겹살', '김치찌개', '제주갈치조림', '흑돼지구이', '해산물']

In [ ]:
food_type = [korean,japanese,chinese,western,bunsik,cafe]

###태그 갯수가 4개이하인 애들은 체크리스트에 담아서 따로 보자

In [ ]:
idx_restaurant_type_matching_info = {0:'한식',1:'일식',2:'중식',3:'양식',4:'분식',5:'카페/베이커리'}

In [ ]:
check_list = [] #분류가 잘 안되는 애들은 체크리스트에 id 담기
entire_category_info = [] #전체 식당데이터에 대한 카테고리 정보
for f in food.values:
  now_id = f[0]
  now_tags = f[11]
  # print(f)
  # print(now_id,now_tags)
  
  num_of_tags = []
  # print('now_tags : ',now_tags)
  for idx, tag_data in enumerate(food_type):
    intersection = (set(tag_data)&set(now_tags)) #타입별 태그와 현재 태그의 교집합 개수
    num_of_tags.append((len(intersection),idx)) #Intersection을 앞에, idx를 뒤에 두기
    # print(intersection)
  num_of_tags.sort()
  max_idx = num_of_tags[-1][1]#오름차순 정렬이므로 맨 우측에 있는게 Max intersection value를 가질것임
  
  if num_of_tags[-1][0] <=2:
    check_list.append(now_id) #곂치는게 별로 없는 경우는 check_list에 담아서 추후 확인하기
  now_type = idx_restaurant_type_matching_info[max_idx] #분류된 타입
  entire_category_info.append(now_type)

In [ ]:
food['type'] = entire_category_info

In [ ]:
food.sample(10)

In [ ]:
len(check_list)

198

In [ ]:
def minus_1(x):
  return x-1
check_list2 = list(map(minus_1,check_list))

In [ ]:
check_list_df = food.iloc[check_list2].copy()

In [ ]:
check_list_df.to_csv("/content/drive/MyDrive/yeoreodigm/data_files/09.28_restaurant_type_checklist.csv")

In [ ]:
check_list_df.columns

Index(['id', 'title', 'roadaddress', 'tag', 'introduction', 'latitude',
       'longitude', 'phoneno', 'image_url', '인기점수', 'popular_tag', 'tag_list',
       'num_of_tag', 'type'],
      dtype='object')

In [ ]:
check_list_df = check_list_df[['id', 'title','tag','introduction','type']]
check_list_df

,id,title,tag,introduction,type
24,25,명랑스낵,"떡볶이,왕새우튀김,흑돼지튀김,생맥주,짜장떡볶이,음식",제주시 한림읍에 위치한 분식집인 명랑스낵은 아늑한 실내에서도 음식을 즐길 수 있으며...,분식
32,33,르 쉬느아 (Le Chinois),"중국음식,딤섬,북경오리,불도장,음식,오리고기,오리로스","2018제주 푸드 앤 와인 페스티벌, 고메위크 선정된 제주신화월드 시그니쳐 중식 레스토랑",한식
46,47,요리하는 목수,"수제버거,햄버거,돈가스,해물짬뽕,음식",30cm 높이의 19단 수제 햄버거와 해물짬뽕을 함께 즐길 수 있는 곳,양식
100,101,쉬림프박스,"커플,혼자,새우,음식,새우버터구이,츄러스",하와이안 스타일의 새우 버터구이가 올라간 밥과 수제 츄러스,카페/베이커리
110,111,제주맥주 양조장,"친구,맑음,사계절,맥주,양조장,브루어리투어,피자,음식,수제맥주",제주도 하면 가장 먼저 떠오르는 로컬맥주인 제주맥주는 서귀포 한림에 양조장을 두고 ...,양식
...,...,...,...,...,...
1593,1594,윈드스톤,"북카페,광령,아몬드라떼,음식,카페",제주시 광령리에 위치한 카페인 윈드스톤은 조용한 동네에서 옛스러운 가옥을 개조하였으...,카페/베이커리
1600,1601,협재온다정,"협재,맑은곰탕,명란,음식,식당",협재해수욕장 인근에 위치한 곰탕전문점인 협재온다정은 흑돼지로만 국물을 우려내고 있으...,양식
1615,1616,라토커피,"제주시청,크림브륄레,카페,음식,식당",제주시청 인근에 위치한 라토커피는 주택을 개조하여 만든 카페이며 곳곳에 아기자기하고...,카페/베이커리
1618,1619,한라축산정육식당 노형점,"정육식당,노형동,함흥식냉면,음식,식당",제주공항 근처 노형동에 위치한 정육식당인 한라축산정육식당은 한돈과 한우를 저렴한 가...,양식


###sort 테스트

In [ ]:
lst=[(1,2),(2,3),(0,4),(2,2)]
lst.sort()
lst

[(0, 4), (1, 2), (2, 2), (2, 3)]

In [ ]:
lst[-1]

(2, 3)

##checklist 노가다 결과 반영하기 

In [2]:
import pandas as pd
import json
import psycopg2
from tqdm import tqdm

In [ ]:
food = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/09.28_restaurant.csv",index_col=0)
food.sample(3)

In [40]:
check_list_apply_result = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/check_list_apply_result.csv",encoding='cp949')
check_list_apply_result.head(3)

,id
0,"33,중식"
1,"101,양식"
2,"118,양식"


In [41]:
check_list_apply_result['check_list_id'] = check_list_apply_result['id'].apply(lambda x: int(x.split(sep=',')[0]))
check_list_apply_result['check_list_type'] = check_list_apply_result['id'].apply(lambda x: x.split(sep=',')[1])

In [42]:
check_list_apply_result.drop('id',axis=1,inplace=True)
check_list_apply_result

,check_list_id,check_list_type
0,33,중식
1,101,양식
2,118,양식
3,133,양식
4,196,양식
...,...,...
67,1584,한식
68,1589,분식
69,1590,한식
70,1601,한식


In [56]:
merge_food = food.merge(right = check_list_apply_result,left_on='id',right_on='check_list_id',how='outer')

In [58]:
merge_food.loc[merge_food['check_list_id'].notnull(),['id','title','type','check_list_type']]

,id,title,type,check_list_type
32,33,르 쉬느아 (Le Chinois),한식,중식
100,101,쉬림프박스,카페/베이커리,양식
117,118,톰톰카레,카페/베이커리,양식
132,133,맥파이 브루어리,카페/베이커리,양식
195,196,카페오름,카페/베이커리,양식
...,...,...,...,...
1583,1584,담다,양식,한식
1588,1589,제주머슴네,양식,분식
1589,1590,임순이네,양식,한식
1600,1601,협재온다정,양식,한식


In [61]:
merge_food[merge_food['id']==33][['id','title','introduction','type','check_list_type']]

,id,title,introduction,type,check_list_type
32,33,르 쉬느아 (Le Chinois),"2018제주 푸드 앤 와인 페스티벌, 고메위크 선정된 제주신화월드 시그니쳐 중식 레스토랑",한식,중식


In [62]:
merge_food.loc[merge_food['check_list_id'].notnull(),['type']] = merge_food.loc[merge_food['check_list_id'].notnull(),['check_list_type']].values

In [63]:
merge_food[merge_food['id']==33][['id','title','introduction','type','check_list_type']]

,id,title,introduction,type,check_list_type
32,33,르 쉬느아 (Le Chinois),"2018제주 푸드 앤 와인 페스티벌, 고메위크 선정된 제주신화월드 시그니쳐 중식 레스토랑",중식,중식


In [64]:
merge_food.columns

Index(['id', 'title', 'roadaddress', 'tag', 'introduction', 'latitude',
       'longitude', 'phoneno', 'image_url', '인기점수', 'popular_tag', 'tag_list',
       'num_of_tag', 'type', 'check_list_id', 'check_list_type'],
      dtype='object')

In [65]:
merge_food = merge_food[['id', 'title', 'roadaddress', 'tag', 'introduction', 'latitude',
       'longitude', 'phoneno', 'image_url', '인기점수', 'popular_tag', 'type']]

##데이터파일 저장 및 DB반영

In [66]:
merge_food.to_csv("/content/drive/MyDrive/yeoreodigm/data_files/09.29_restaurant.csv")

In [68]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

In [69]:
db = load_db()
cursor = db.cursor()

In [82]:
for now in tqdm(merge_food.values):
  now_id = now[0]
  now_type = now[-1]
  sql = f"update restaurant set type = \'{now_type}\' where restaurant_id = {now_id}"
  cursor.execute(sql)

100%|██████████| 1623/1623 [04:48<00:00,  5.62it/s]


AttributeError: ignored

In [83]:
db.commit()